### データセットをトレーニングデータセットとテストデータセットに分割する

Wineデータセットを用い、前処理を行った後次元数を減らすための特徴選択の手法を見ていく

In [15]:
import pandas as pd
import numpy as np
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [16]:
from sklearn.cross_validation import train_test_split
# X:特徴量 y: クラスラベル
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 特徴量の尺度を揃える

一般的な手法は__正規化(normalization)__と__標準化(standardization)__の２つ。

#### 正規化

特徴量を[0,1]の範囲にスケーリングし直すこと。

$$ x_{norm}^{(i)} = \frac{x^{(i)} - x_{min}}{x_{max} - x_{min}} $$

In [20]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)
print('正規化前')
print(X_train[0])
print('正規化後')
print(X_train_norm[0])

正規化前
[  1.37100000e+01   1.86000000e+00   2.36000000e+00   1.66000000e+01
   1.01000000e+02   2.61000000e+00   2.88000000e+00   2.70000000e-01
   1.69000000e+00   3.80000000e+00   1.11000000e+00   4.00000000e+00
   1.03500000e+03]
正規化後
[ 0.72043011  0.20378151  0.53763441  0.30927835  0.33695652  0.54316547
  0.73700306  0.25        0.40189873  0.24068768  0.48717949  1.          0.5854251 ]


#### 標準化

平均値0, 標準偏差1となるように変換する。以下の点で正規化より優れている。

* 特徴量の列は正規分布に従うため、重みを学習しやすくなる
* 外れ値に関する有益な情報が維持されるため、外れ値の影響を受けにくい

$$ x_{std}^{(i)} = \frac{x^{(i)} - \mu_x}{\sigma_x} $$

* \\( \mu_x \\)：特徴量の列の平均値
* \\( \sigma_x \\)：対応する標準偏差

In [22]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

print('標準化前')
print(X_train[0])
print('標準化後')
print(X_train_std[0])

標準化前
[  1.37100000e+01   1.86000000e+00   2.36000000e+00   1.66000000e+01
   1.01000000e+02   2.61000000e+00   2.88000000e+00   2.70000000e-01
   1.69000000e+00   3.80000000e+00   1.11000000e+00   4.00000000e+00
   1.03500000e+03]
標準化後
[ 0.91083058 -0.46259897 -0.01142613 -0.82067872  0.06241693  0.58820446
  0.93565436 -0.7619138   0.13007174 -0.51238741  0.65706596  1.94354495
  0.93700997]
